In [ ]:
%pip install tensorflow keras --quiet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

RANDOM_STATE = 0

Exercice 1 – Perceptron Multicouche avec SciKitLearn (Rappel)

1) Charger le dataset IRIS. Utilisez un pairplot (bibliothèque seaborn) pour visualiser et analyser les données.

In [ ]:
from sklearn.datasets import load_iris
import seaborn as sns

iris = load_iris(as_frame=True).frame
sns.pairplot(data=iris, hue="target")

2) Séparez les données en 4 matrices : x_train, x_test, y_train, y_test. Les données d'entrée sont sur les 4 premières colonnes et les données de sortie sont sur la dernière colonne. Le rapport train/test doit être de 75/25.

In [ ]:
from sklearn.model_selection import train_test_split

X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(
    iris.iloc[:, :4],
    iris.iloc[:, -1],
    test_size=0.25,
    random_state=RANDOM_STATE,
)

print(X_train_iris.shape, X_test_iris.shape, y_train_iris.shape, y_test_iris.shape)


3) En utilisant SKLearn, construire un classificateur logistique sur l'ensemble de données d'entraînement. Évaluer le modèle généré sur l'ensemble de données de test et affichez le score de prédiction. Que pensez-vous de ce score ?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

log_reg = LogisticRegression(random_state=RANDOM_STATE)
log_reg.fit(X_train_iris, y_train_iris)
y_pred_iris = log_reg.predict(X_test_iris)
print(accuracy_score(y_test_iris, y_pred_iris))
print(classification_report(y_test_iris, y_pred_iris))
print(log_reg.n_iter_)

4) Transformez les données pour les rendre utilisables par un réseau de neurones : la sortie doit être une matrice Nxn (n étant le nombre de classes du problème et N le nombre d'observations) au lieu d'un vecteur Nx1. Chaque colonne doit contenir un 1 si l'observation est de cette classe et zéro sinon. Regarder la fonction to_categorical.

In [ ]:
from keras.utils import to_categorical

y_train_iris_cat = to_categorical(y_train_iris)
y_test_iris_cat = to_categorical(y_test_iris)
print(y_train_iris_cat[:5])
print(y_test_iris_cat[:5])

5) Toujours en utilisant SKLearn, créer un perceptron pour classifier le jeu de données IRIS. Le réseau doit contenir une couche cachée de 16 neurones. Vous pouvez modifier les hyper-paramètres (solveur, fonction d'activation, taux d'apprentissage, etc.) du réseau comme vous le souhaitez.

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    hidden_layer_sizes=(16,),
    random_state=RANDOM_STATE,
    max_iter=5000,
)

6) Entraînez le réseau sur l'ensemble de données d'entraînement et affichez le score sur les ensembles de données d'entraînement et de test. 

In [ ]:
mlp.fit(X_train_iris, y_train_iris_cat)
y_pred_iris_mlp = mlp.predict(X_test_iris)
print(accuracy_score(y_test_iris_cat, y_pred_iris_mlp))
print(classification_report(y_test_iris_cat, y_pred_iris_mlp))

print(mlp.n_iter_)

Exercice 2 – Perceptron Multicouche avec Keras

1) À l'aide de Keras, créez un perceptron construit comme suit :

a. Une couche dense de 16 neurones connectée à une couche d'entrée de 4 neurones. La fonction d'activation de cette couche doit être une sigmoïde.

In [ ]:
import keras

inputs = keras.Input(shape=(4,))
hidden = keras.layers.Dense(16, activation="sigmoid")(inputs)

b. Une couche de sortie de 3 neurones avec une fonction d'activation softmax.

In [ ]:
outputs = keras.layers.Dense(3, activation="softmax")(hidden)

2) Utiliser la fonction de perte categorical_crossentropy (perte logarithmique), l'optimiseur Adam et activer le précalcul de la métrique de précision.

In [ ]:
model_iris = keras.Model(inputs=inputs, outputs=outputs)
model_iris.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

3) Entraîner le réseau pendant 100 epochs.

In [ ]:
model_iris.fit(X_train_iris, y_train_iris_cat, epochs=100, verbose=0)

4) Comparer la précision de la classification obtenue avec la régression logistique de l'exercice 2.

In [ ]:
model_iris.evaluate(X_test_iris, y_test_iris_cat, verbose=1)

Exercice 3 – Adapter un réseau à un problème donné

1) Charger le dataset MNist et visualiser les données chargées. Affichez les premières images du jeu de données. Que contient ce dataset et quel est son objectif ?

In [ ]:
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = (
    keras.datasets.mnist.load_data()
)

plt.imshow(x_train_mnist[0].reshape(28, 28), cmap="gray_r")
plt.title(y_train_mnist[0])
plt.axis("off")
plt.show()

2) Modifier les données pour :

a. Linéariser les données en deux dimensions.

In [ ]:
x_train_mnist = x_train_mnist.reshape(-1, 28 * 28)
x_test_mnist = x_test_mnist.reshape(-1, 28 * 28)

print(x_train_mnist.shape, x_test_mnist.shape)

b. Normaliser les données

In [ ]:
x_train_mnist = x_train_mnist / 255.0
x_test_mnist = x_test_mnist / 255.0

c. Transformer le vecteur de classes pour qu'il soit utilisable par un réseau de neurones.

In [ ]:
y_train_mnist_cat = to_categorical(y_train_mnist)
y_test_mnist_cat = to_categorical(y_test_mnist)

print(y_train_mnist_cat[:5])
print(y_test_mnist_cat[:5])

3) Entraîner l'architecture de l'exercice précédent à ce nouveau problème. Attention à la taille des couches d'entrée et de sortie !

In [ ]:
model_mnist = keras.Sequential(
    [
        keras.Input(shape=(28 * 28,)),
        keras.layers.Dense(16, activation="sigmoid"),
        keras.layers.Dense(10, activation="softmax"),
    ]
)
model_mnist.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)
model_mnist.fit(x_train_mnist, y_train_mnist_cat)
model_mnist.evaluate(x_test_mnist, y_test_mnist_cat)

4) Entraîner le réseau de quelques epochs avec une taille de batch de 128. Observez le score du réseau. Que pensez-vous de ce résultat ?

In [ ]:
model_mnist.fit(x_train_mnist, y_train_mnist_cat, epochs=10, batch_size=128)
model_mnist.evaluate(x_test_mnist, y_test_mnist_cat)

Exercice 4 – Créer des réseaux plus profonds

1. Charger, afficher et formater le jeu de données Cifar10

In [ ]:
(X_train_cifar10, y_train_cifar10), (X_test_cifar10, y_test_cifar10) = (
    keras.datasets.cifar10.load_data()
)

fig, axs = plt.subplots(1, 10, figsize=(10, 5))

for i in range(10):
    axs[i].imshow(X_train_cifar10[i].reshape(32, 32, 3), cmap="gray_r")
    axs[i].set_title(y_train_cifar10[i])
    axs[i].axis("off")

plt.tight_layout()
plt.show()

2. Adapter  le  perceptron  de  l'exercice  précédent.  Entraîner,  évaluer  et  afficher  les 
erreurs de classification. Essayez  de modifier doucement le réseau (pas de nouvelle 
couche  mais  une  couche  cachée  plus  grande,  des fonctions  d'activation  différentes, 
etc.)  et  évaluer  ces  différents  réseaux.  Que  pensez-vous  de  la  prédiction  que  vous 
obtenez ?

In [ ]:
X_train_cifar10 = X_train_cifar10.reshape(-1, 32 * 32 * 3) / 255
X_test_cifar10 = X_test_cifar10.reshape(-1, 32 * 32 * 3) / 255

X_validation_cifar10 = X_train_cifar10[-5000:]
y_validation_cifar10 = y_train_cifar10[-5000:]
X_train_cifar10 = X_train_cifar10[:-5000]
y_train_cifar10 = y_train_cifar10[:-5000]

y_train_cifar10_cat = to_categorical(y_train_cifar10)
y_test_cifar10_cat = to_categorical(y_test_cifar10)

model_cifar10 = keras.Sequential(
    [
        keras.Input(shape=(32 * 32 * 3,)),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(10, activation="softmax"),
    ]
)
model_cifar10.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)
model_cifar10.fit(
    X_train_cifar10,
    y_train_cifar10_cat,
    epochs=0,
    validation_data=(X_validation_cifar10, y_validation_cifar10),
)
model_cifar10.evaluate(X_test_cifar10, y_test_cifar10_cat)

In [ ]:
pred = model_cifar10.predict(X_test_cifar10, batch_size=256).argmax(axis=1)
wrong_idx = np.where(pred != y_test_cifar10)[0][:25]

plt.figure(figsize=(10, 10))
for i, idx in enumerate(wrong_idx):
    plt.subplot(5, 5, i + 1)
    plt.imshow(X_test_cifar10[idx].reshape(32, 32, 3) * 255)
    plt.title(f"Ground truth:{y_test_cifar10[idx]}\nPrediction:{pred[idx]}")
    plt.axis("off")
plt.tight_layout()
plt.show()


3. Construire le perceptron multicouche suivant :

a. Première couche cachée de 512 neurones densément connectés à la couche 
d'entrée, fonction d'activation relu, dropout de 0,2 et normalisation par lots.

b. Deuxième couche cachée de 512 neurones densément connectés à la couche 
d'entrée, fonction d'activation Relu, normalisation par lot et dropout de 0,2.

In [ ]:
model_cifar10_2 = keras.Sequential(
    [
        keras.Input(shape=(32 * 32 * 3,)),
        #
        keras.layers.Dense(512),
        keras.layers.ReLU(),
        keras.layers.Dropout(0.2),
        keras.layers.BatchNormalization(),
        #
        keras.layers.Dense(512),
        keras.layers.ReLU(),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        #
        keras.layers.Dense(10, activation="softmax"),
    ]
)

4. Exécuter 50 epochs d'apprentissage et évaluer le réseau obtenu. Que pensez-vous du 
score de prédiction ?

In [ ]:
model_cifar10_2.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)
history_cifar10_2 = model_cifar10_2.fit(
    X_train_cifar10,
    y_train_cifar10,
    epochs=50,
    verbose=0,
    batch_size=128,
    validation_data=(X_validation_cifar10, y_validation_cifar10),
)
train_loss_cifar10_2, train_acc_cifar10_2 = model_cifar10_2.evaluate(
    X_train_cifar10,
    y_train_cifar10,
    verbose=0,
)
test_loss_cifar10_2, test_acc_cifar10_2 = model_cifar10_2.evaluate(
    X_test_cifar10,
    y_test_cifar10,
    verbose=0,
)
print(f"Train  — loss: {train_loss_cifar10_2:.4f}, acc: {train_acc_cifar10_2:.4f}")
print(f"Test   — loss: {test_loss_cifar10_2:.4f}, acc: {test_acc_cifar10_2:.4f}")

In [ ]:
plt.figure()
plt.plot(history_cifar10_2.history["loss"])
plt.plot(history_cifar10_2.history["val_loss"])
plt.legend(["train", "validation"])
plt.title("Loss")
plt.xlabel("epoch")
plt.figure()
plt.plot(history_cifar10_2.history["accuracy"])
plt.plot(history_cifar10_2.history["val_accuracy"])
plt.legend(["train", "validation"])
plt.title("Accuracy")
plt.xlabel("epoch")
plt.show()

5. Enregistrer votre modèle sur votre disque dur et calculer la précision et la perte pour 
chaque epoch d'entraînement.

6. Pensez-vous pouvoir faire mieux ? Si oui, construisez l'architecture ! 

Exercice 5 – Réseau de neurones

1) Construire l’architecture générale du réseau. Créer les classes : 
a. Neuron  contenant une valeur, l’ensemble des synapses entrant et sortant, 
une fonction d’agrégation et une fonction d’activation  
b. Synapse contenant le poids synaptique et le neurone entrant et le sortant 
c. MultiLayerPerceptron  contenant  une  liste  de  Neuron  organisés  en 
couche et les paramètres nécessaires à l’apprentissage

In [ ]:
class Neuron:
    def __init__(self, func_aggregation=None, func_activation=None):
        self.value: float = 0.0
        self.synapses_in: list[Synapse] = []
        self.synapses_out: list[Synapse] = []
        self.func_aggregation: callable = func_aggregation
        self.func_activation: callable = func_activation


class Synapse:
    def __init__(self, pre: Neuron, post: Neuron, w: float = 0.0):
        self.pre = pre
        self.post = post
        self.w = w

2) Implémenter les classes ci-dessus pour que la classe 
MultiLayerPerceptron (seule visible pour l’utilisateur final) puisse : 
a. __init__ : Créer  un  perceptron  à  partir  d’un  tableau  représentant  le 
nombre de neurones à créer dans chaque couche. Par exemple si on passe [3, 
5, 3, 2], il faut construire un réseau contenant 1 couche d’entrée contenant 3 
neurones,  2  couches  cachées  contenant  respectivement  5  et  3  neurones  et 
une couche de sortie contenant 2 neurones. 
b. randomize_weights :  Initialiser  aléatoirement  tous  les  poids  du  réseau 
entre [-1, 1]. 
c. predict Prédire un résultat à partir un exemple d’entrainement (1 ligne du 
dataset). 
d. update_weights : Mettre à jours les poids synaptiques en utilisant 
l’algorithme de Hebb vu en cours. 
e. train : Entrainer un réseau sur un dataset donné.